In [ ]:
import numpy as np
import pandas as pd
import lightgbm as lgbm
import pandas_profiling as pp
from sklearn import preprocessing

In [ ]:
df = pd.read_csv("/content/train.csv")
df

,MEAN_RR,MEDIAN_RR,SDRR,RMSSD,SDSD,SDRR_RMSSD,HR,pNN25,pNN50,SD1,...,HF,HF_PCT,HF_NU,TP,LF_HF,HF_LF,sampen,higuci,datasetId,condition
0,885.157845,853.763730,140.972741,15.554505,15.553371,9.063146,69.499952,11.133333,0.533333,11.001565,...,15.522603,0.421047,1.514737,3686.666157,65.018055,0.015380,2.139754,1.163485,2,no stress
1,939.425371,948.357865,81.317742,12.964439,12.964195,6.272369,64.363150,5.600000,0.000000,9.170129,...,2.108525,0.070133,0.304603,3006.487251,327.296635,0.003055,2.174499,1.084711,2,interruption
2,898.186047,907.006860,84.497236,16.305279,16.305274,5.182201,67.450066,13.066667,0.200000,11.533417,...,13.769729,0.512671,1.049528,2685.879461,94.280910,0.010607,2.135350,1.176315,2,interruption
3,881.757865,893.460030,90.370537,15.720468,15.720068,5.748591,68.809562,11.800000,0.133333,11.119476,...,18.181913,0.529387,1.775294,3434.520980,55.328701,0.018074,2.178341,1.179688,2,no stress
4,809.625331,811.184865,62.766242,19.213819,19.213657,3.266724,74.565728,20.200000,0.200000,13.590641,...,48.215822,1.839473,3.279993,2621.175204,29.487873,0.033912,2.221121,1.249612,2,no stress
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369284,721.396910,721.533965,36.377559,9.609941,9.609936,3.785409,83.384647,0.933333,0.000000,6.797519,...,38.227175,5.886975,10.880317,649.351702,8.190908,0.122087,2.096850,1.314910,2,no stress
369285,984.266492,978.622945,74.918433,13.762274,13.761705,5.443754,61.314243,6.333333,0.200000,9.734243,...,2.573834,0.112964,0.357547,2278.456746,278.683532,0.003588,2.131264,1.175975,2,no stress
369286,1025.499743,1024.968400,95.309200,23.051254,23.050395,4.134664,59.028594,30.400000,2.066667,16.304530,...,4.287216,0.098983,0.180070,4331.259564,554.340761,0.001804,2.215643,1.140232,2,no stress
369287,798.123167,803.559610,78.449897,16.736686,16.736657,4.687302,75.978628,10.000000,1.466667,11.838553,...,46.636158,1.591647,3.522995,2930.056129,27.384942,0.036516,2.091628,1.181937,2,no stress


In [ ]:
df1 = df.iloc [:,:-1]
df1

,MEAN_RR,MEDIAN_RR,SDRR,RMSSD,SDSD,SDRR_RMSSD,HR,pNN25,pNN50,SD1,...,LF_NU,HF,HF_PCT,HF_NU,TP,LF_HF,HF_LF,sampen,higuci,datasetId
0,885.157845,853.763730,140.972741,15.554505,15.553371,9.063146,69.499952,11.133333,0.533333,11.001565,...,98.485263,15.522603,0.421047,1.514737,3686.666157,65.018055,0.015380,2.139754,1.163485,2
1,939.425371,948.357865,81.317742,12.964439,12.964195,6.272369,64.363150,5.600000,0.000000,9.170129,...,99.695397,2.108525,0.070133,0.304603,3006.487251,327.296635,0.003055,2.174499,1.084711,2
2,898.186047,907.006860,84.497236,16.305279,16.305274,5.182201,67.450066,13.066667,0.200000,11.533417,...,98.950472,13.769729,0.512671,1.049528,2685.879461,94.280910,0.010607,2.135350,1.176315,2
3,881.757865,893.460030,90.370537,15.720468,15.720068,5.748591,68.809562,11.800000,0.133333,11.119476,...,98.224706,18.181913,0.529387,1.775294,3434.520980,55.328701,0.018074,2.178341,1.179688,2
4,809.625331,811.184865,62.766242,19.213819,19.213657,3.266724,74.565728,20.200000,0.200000,13.590641,...,96.720007,48.215822,1.839473,3.279993,2621.175204,29.487873,0.033912,2.221121,1.249612,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369284,721.396910,721.533965,36.377559,9.609941,9.609936,3.785409,83.384647,0.933333,0.000000,6.797519,...,89.119683,38.227175,5.886975,10.880317,649.351702,8.190908,0.122087,2.096850,1.314910,2
369285,984.266492,978.622945,74.918433,13.762274,13.761705,5.443754,61.314243,6.333333,0.200000,9.734243,...,99.642453,2.573834,0.112964,0.357547,2278.456746,278.683532,0.003588,2.131264,1.175975,2
369286,1025.499743,1024.968400,95.309200,23.051254,23.050395,4.134664,59.028594,30.400000,2.066667,16.304530,...,99.819930,4.287216,0.098983,0.180070,4331.259564,554.340761,0.001804,2.215643,1.140232,2
369287,798.123167,803.559610,78.449897,16.736686,16.736657,4.687302,75.978628,10.000000,1.466667,11.838553,...,96.477005,46.636158,1.591647,3.522995,2930.056129,27.384942,0.036516,2.091628,1.181937,2


In [ ]:
label = df.iloc[:,-1:]
label

,condition
0,no stress
1,interruption
2,interruption
3,no stress
4,no stress
...,...
369284,no stress
369285,no stress
369286,no stress
369287,no stress


In [ ]:
label['condition'][label['condition']=="no stress"] = 1
label['condition'][label['condition']=="time pressure"] = 2
label['condition'][label['condition']=="interruption"] = 3

/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:8870: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [ ]:
label = label.astype('int')

In [ ]:
import pickle
params = {'objective':'multiclass',
          'booster_type':'gbdt',
          'max_depth':10,
          'learning_rate':0.05,
          'metric':'multi_logloss',
          'num_class':4

    }


dtrain = lgbm.Dataset(df1,label)
model = lgbm.train(params,dtrain,num_boost_round=100,valid_sets=[dtrain])

filename = '/content/panicpred.sav'
pickle.dump(model, open(filename, 'wb'))

#model.save_model('/content/saved_adult_model1.txt')

[1]	training's multi_logloss: 0.955568
[2]	training's multi_logloss: 0.920791
[3]	training's multi_logloss: 0.888583
[4]	training's multi_logloss: 0.857811
[5]	training's multi_logloss: 0.82991
[6]	training's multi_logloss: 0.802722
[7]	training's multi_logloss: 0.777613
[8]	training's multi_logloss: 0.752828
[9]	training's multi_logloss: 0.730564
[10]	training's multi_logloss: 0.708638
[11]	training's multi_logloss: 0.688387
[12]	training's multi_logloss: 0.668355
[13]	training's multi_logloss: 0.64977
[14]	training's multi_logloss: 0.631932
[15]	training's multi_logloss: 0.615154
[16]	training's multi_logloss: 0.599557
[17]	training's multi_logloss: 0.58445
[18]	training's multi_logloss: 0.569853
[19]	training's multi_logloss: 0.555656
[20]	training's multi_logloss: 0.542414
[21]	training's multi_logloss: 0.529262
[22]	training's multi_logloss: 0.516817
[23]	training's multi_logloss: 0.504932
[24]	training's multi_logloss: 0.493092
[25]	training's multi_logloss: 0.481793
[26]	trainin

In [ ]:
import numpy as np
import statistics

def get_values(hr):
  hr_data = [hr,hr-5,hr+5]
  # Calculate the RR intervals from the heart rate data (in milliseconds)
  rr_data = [60000/hr for hr in hr_data]
  # Calculate the mean RR interval
  get_values.mean_rr = np.mean(rr_data)
  # Calculate the median RR interval
  get_values.median_rr = np.median(rr_data)

  #return hr,mean_rr, median_rr

In [ ]:
df_test = pd.read_csv('/content/test.csv')

In [ ]:
df_test = df_test['HR']
df_test.head()

0    84.121868
1    71.478642
2    63.874293
3    74.330531
4    82.092049
Name: HR, dtype: float64

In [ ]:
#x_test = [3.900e+01, 7.000e+00, 1.300e+01, 4.000e+00, 1.000e+00, 0.000e+00,4.000e+00, 1.000e+00, 2.174e+03, 0.000e+00, 4.000e+01, 3.900e+01]
#x_test = np.array(x_test).reshape((1,-1))
#print(x_test)
#y = [843.5386334, 844.40793, 58.49942906, 19.29888005, 19.29879537, 3.031234399, 71.47864202, 21, 0.2, 13.65086316, 81.59669316, -0.445805949, -0.144911278, 6.13e-05, -0.001542892, 0.022969015, 0.011969554, 0.011969553, 1.918953216, -0.445805949, -0.144911278, 765.5184725, 32.45256008, 1566.866135, 66.424024, 98.336849, 26.50008618, 1.12341592, 1.663150996, 2358.884694, 59.12683168, 0.016912795, 2.217274814, 1.250055937, 2]

hr = df_test

for hr in hr:
  ans_get_values = get_values(hr)
  mean_rr = get_values.mean_rr
  median_rr = get_values.median_rr
  y1 = pd.DataFrame({'HR': [hr], 'MEAN_RR':[mean_rr], 'MEDIAN_RR':[median_rr]})
  x_test = np.array(y1).reshape((1,-1))
  pred=model.predict(x_test)
  #argmax() method 
  pred = [np.argmax(line) for line in pred]
  print(pred)

  #using precision score for error metrics
  #precision_score(pred,average=None).mean()

Streaming output truncated to the last 5000 lines.
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[2]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[2]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[

### Lambda Ref

In [ ]:
def get_model():
    bucket= boto3.resource('s3').Bucket('deploy-lgbm')
    bucket.download_file('model/saved_model.txt','/tmp/test_model.txt')
    model= lightgbm.Booster(model_file='/tmp/test_model.txt')
    return model

In [ ]:
def predict(event):
    sample =event['body']
    model = get_model()
    result = model.predict(sample)
    return result

In [ ]:
def lambda_handler(event,context):
    result = predict(event)
    return {'StatusCode':200,
    'body':result[0]}